In [14]:
import requests
import pandas as pd
import logging
import time 
from concurrent.futures import ThreadPoolExecutor, as_completed

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Número máximo de requisições concorrentes (threads)
MAX_CONCURRENT_REQUESTS = 5 # Ajuste conforme necessário (5-10 é um bom começo)

def buscar_e_processar_shape_linha(codigo_linha_str):
    """
    Busca os dados do shape (trajeto) de uma linha de ônibus específica da API da URBS.
    O codigo_linha_str deve ser uma string (ex: "010", "203", "666").
    Retorna um DataFrame com colunas: SHP, Latitude, Longitude, COD.
    """
    codigo_linha_formatado = codigo_linha_str.zfill(3)
    url = f'https://transporteservico.urbs.curitiba.pr.gov.br/getShapeLinha.php?linha={codigo_linha_formatado}&c=821f0'
    try:
        response = requests.get(url, timeout=15)
        response.raise_for_status()
        dados_json = response.json()

        if not dados_json:
            # logging.info(f"Nenhum dado JSON de SHAPE retornado para a linha {codigo_linha_str}.")
            return None
        
        if not isinstance(dados_json, list) or not all(isinstance(item, dict) for item in dados_json):
            logging.warning(f"Formato de dados de SHAPE inesperado para a linha {codigo_linha_str}. Dados: {str(dados_json)[:200]}...")
            return None
        
        if not dados_json: # Segurança extra
             return None

        df = pd.DataFrame.from_records(dados_json)

        # Colunas esperadas da API getShapeLinha
        required_cols_from_api = {'SHP', 'LAT', 'LON'} 
        if not required_cols_from_api.issubset(df.columns):
            # A API getShapeLinha também retorna 'COD' no JSON, mas vamos adicionar o nosso para consistência
            # Se a API retornar COD, podemos usá-lo ou compará-lo. Por agora, vamos focar em SHP, LAT, LON.
            # Se a API retornar COD, a linha abaixo pode ser ajustada.
            # O exemplo fornecido na pergunta mostra que a API retorna COD, então vamos incluir.
            if 'COD' in df.columns: # Se a API já retorna COD, ótimo.
                required_cols_from_api.add('COD') # Adiciona à verificação
            
            if not required_cols_from_api.issubset(df.columns):
                 logging.warning(f"Colunas de SHAPE esperadas ('SHP', 'LAT', 'LON') ausentes da API para a linha {codigo_linha_str}. Colunas recebidas: {df.columns.tolist()}.")
                 return None


        df['Latitude'] = df['LAT'].str.replace(',', '.').astype(float)
        df['Longitude'] = df['LON'].str.replace(',', '.').astype(float)
        
        # A API getShapeLinha já retorna uma coluna 'COD' no JSON.
        # Vamos garantir que ela exista e seja do tipo string para consistência.
        if 'COD' in df.columns:
            df['COD'] = df['COD'].astype(str)
        else:
            # Se a API não retornar COD (improvável baseado no exemplo), adicionamos o que usamos na busca
            df['COD'] = codigo_linha_str
        
        # Garante que SHP seja string
        if 'SHP' in df.columns:
            df['SHP'] = df['SHP'].astype(str)
        else: # Caso SHP não venha, o que seria um erro da API para esta chamada
            logging.error(f"Coluna 'SHP' ausente nos dados da API para a linha {codigo_linha_str}")
            return None


        colunas_desejadas = ['SHP', 'Latitude', 'Longitude', 'COD']
        return df[colunas_desejadas]

    except requests.exceptions.JSONDecodeError:
        logging.error(f"Erro ao decodificar JSON de SHAPE para a linha {codigo_linha_str}. URL: {url}. Resposta: {response.text[:200]}...")
    except requests.exceptions.HTTPError as http_err:
        logging.warning(f"Erro HTTP {http_err.response.status_code} para SHAPE da linha {codigo_linha_str} (URL: {url}).")
    except requests.RequestException as e:
        logging.error(f"Erro na requisição de SHAPE para a linha {codigo_linha_str} (URL: {url}): {e}")
    except KeyError as e: 
        logging.error(f"Coluna esperada não encontrada ao processar SHAPE da linha {codigo_linha_str}: {e}. Dados: {dados_json if 'dados_json' in locals() else 'N/A'}")
    except Exception as e:
        logging.error(f"Erro inesperado ao processar SHAPE da linha {codigo_linha_str} (URL: {url}): {e}", exc_info=False)
    return None

def coletar_shapes_de_todas_as_linhas_com_threads(lista_codigos_linhas, max_workers=MAX_CONCURRENT_REQUESTS):
    """
    Coleta dados de SHAPE de todas as linhas de ônibus fornecidas na lista usando threads.
    Retorna um DataFrame único com todos os dados ou um DataFrame vazio.
    """
    todos_os_shapes_coletados_dfs = []
    logging.info(f"Iniciando coleta de SHAPES com até {max_workers} threads para {len(lista_codigos_linhas)} linhas únicas.")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_linha = {executor.submit(buscar_e_processar_shape_linha, codigo_linha): codigo_linha for codigo_linha in lista_codigos_linhas}
        
        processed_count = 0
        for future in as_completed(future_to_linha):
            codigo_linha = future_to_linha[future]
            processed_count += 1
            try:
                df_shape_da_linha = future.result()
                if df_shape_da_linha is not None and not df_shape_da_linha.empty:
                    todos_os_shapes_coletados_dfs.append(df_shape_da_linha)
                    logging.info(f"SHAPE coletado para a linha {codigo_linha} ({len(df_shape_da_linha)} pontos). Progresso: {processed_count}/{len(lista_codigos_linhas)}")
                else:
                    logging.info(f"Nenhum SHAPE válido para linha {codigo_linha}. Progresso: {processed_count}/{len(lista_codigos_linhas)}")
            except Exception as exc:
                logging.error(f"Linha {codigo_linha} (SHAPE) gerou uma exceção ao obter resultado da thread: {exc}", exc_info=False)
                logging.info(f"Progresso: {processed_count}/{len(lista_codigos_linhas)}")

    if not todos_os_shapes_coletados_dfs:
        logging.warning("Nenhum SHAPE foi coletado de nenhuma das linhas especificadas usando threads.")
        return pd.DataFrame() 

    df_consolidado = pd.concat(todos_os_shapes_coletados_dfs, ignore_index=True)
    logging.info(f"Total de {len(df_consolidado)} pontos de SHAPE coletados de todas as linhas com threads (antes da remoção de duplicatas).")
    
    return df_consolidado

def iniciar_coleta_shapes_para_dataframe(caminho_arquivo_csv_linhas):
    logging.info(f"Iniciando processo de coleta de SHAPES a partir do arquivo: {caminho_arquivo_csv_linhas}")
    try:
        df_arquivo_linhas = pd.read_csv(caminho_arquivo_csv_linhas)
        
        if 'COD_Linha' not in df_arquivo_linhas.columns:
            logging.error(f"A coluna 'COD_Linha' não foi encontrada no arquivo CSV: {caminho_arquivo_csv_linhas}.")
            return
        
        # Converte para string para garantir que "010" seja tratado como string
        linhas_unicas_para_coleta = df_arquivo_linhas['COD_Linha'].astype(str).unique().tolist()
        
        if not linhas_unicas_para_coleta:
            logging.warning("Nenhum código de linha único para processar encontrado no arquivo CSV.")
            return

        logging.info(f"Encontrados {len(linhas_unicas_para_coleta)} códigos de linha únicos para processamento de SHAPES: {linhas_unicas_para_coleta[:10]}... (mostrando até 10)")

        df_todos_os_shapes = coletar_shapes_de_todas_as_linhas_com_threads(linhas_unicas_para_coleta)

        if df_todos_os_shapes.empty:
            logging.info("Nenhum SHAPE foi coletado após processar todas as linhas com threads. Encerrando.")
            return

        registros_antes_dedup = len(df_todos_os_shapes)
        # Para shapes, cada ponto é definido por COD, SHP, Latitude, Longitude.
        # A ordem dos pontos dentro de um SHP é importante e deve ser preservada pela API.
        # drop_duplicates aqui removeria pontos idênticos se existirem.
        df_todos_os_shapes.drop_duplicates(subset=['COD', 'SHP', 'Latitude', 'Longitude'], keep='first', inplace=True)
        registros_depois_dedup = len(df_todos_os_shapes)
        logging.info(f"Remoção de duplicatas (pontos de SHAPE idênticos): {registros_antes_dedup - registros_depois_dedup} registros duplicados removidos.")
        logging.info(f"Número final de registros de SHAPE únicos: {registros_depois_dedup}")

        logging.info("Amostra dos dados finais de SHAPE coletados e processados (DataFrame 'df_todos_os_shapes'):")
        print(df_todos_os_shapes.head())
        
        nome_arquivo_saida = "dados_shapes_onibus_urbs_consolidado_threads.csv"
        df_todos_os_shapes.to_csv(nome_arquivo_saida, index=False, encoding='utf-8-sig')
        logging.info(f"DataFrame de SHAPES consolidado e limpo salvo em: '{nome_arquivo_saida}'")

    except FileNotFoundError:
        logging.error(f"Arquivo de linhas não encontrado em: {caminho_arquivo_csv_linhas}")
    except pd.errors.EmptyDataError:
        logging.error(f"O arquivo CSV de linhas ({caminho_arquivo_csv_linhas}) está vazio ou mal formatado.")
    except Exception as e:
        logging.error(f"Um erro geral ocorreu durante o processamento de SHAPES: {e}", exc_info=True)
    finally:
        logging.info("Processamento de SHAPES finalizado.")

# --- Para Executar o Script ---
if __name__ == '__main__':
    # Certifique-se que o arquivo Linha.csv está no mesmo diretório
    # ou forneça o caminho completo.
    # A coluna com os códigos das linhas no CSV deve se chamar 'COD_Linha'.
    caminho_do_arquivo_csv_com_linhas = 'Linha.csv' 
    
    iniciar_coleta_shapes_para_dataframe(caminho_do_arquivo_csv_com_linhas)


2025-05-26 17:37:11,133 - INFO - Iniciando processo de coleta de SHAPES a partir do arquivo: Linha.csv
2025-05-26 17:37:11,141 - INFO - Encontrados 276 códigos de linha únicos para processamento de SHAPES: ['10', '11', '20', '21', '22', '23', '30', '40', '50', '60']... (mostrando até 10)
2025-05-26 17:37:11,142 - INFO - Iniciando coleta de SHAPES com até 5 threads para 276 linhas únicas.
2025-05-26 17:37:11,361 - INFO - SHAPE coletado para a linha 20 (982 pontos). Progresso: 1/276
2025-05-26 17:37:11,371 - INFO - SHAPE coletado para a linha 11 (649 pontos). Progresso: 2/276
2025-05-26 17:37:11,447 - INFO - SHAPE coletado para a linha 10 (665 pontos). Progresso: 3/276
2025-05-26 17:37:11,517 - INFO - SHAPE coletado para a linha 22 (973 pontos). Progresso: 4/276
2025-05-26 17:37:11,558 - INFO - SHAPE coletado para a linha 21 (1025 pontos). Progresso: 5/276
2025-05-26 17:37:11,679 - INFO - SHAPE coletado para a linha 23 (961 pontos). Progresso: 6/276
2025-05-26 17:37:11,819 - INFO - SHAPE

    SHP   Latitude  Longitude  COD
0  4091 -25.492281 -49.293103  020
1  4091 -25.491527 -49.293194  020
2  4091 -25.491572 -49.293684  020
3  4091 -25.491621 -49.293843  020
4  4091 -25.491644 -49.295450  020


2025-05-26 17:37:28,366 - INFO - DataFrame de SHAPES consolidado e limpo salvo em: 'dados_shapes_onibus_urbs_consolidado_threads.csv'
2025-05-26 17:37:28,366 - INFO - Processamento de SHAPES finalizado.


In [15]:
import pandas as pd
df_pontos_rota = pd.read_csv('dados_shapes_onibus_urbs_consolidado_threads.csv')

In [17]:
df_pontos_rota['COD'].unique()

array([ 20,  11,  10,  22,  21,  23,  60,  30,  50,  40, 150, 164, 160,
       166, 168, 169, 170, 171, 175, 176, 177, 182, 183, 184, 189, 203,
       181, 188, 207, 205, 211, 212, 210, 213, 214, 206, 224, 222, 226,
       225, 209, 216, 229, 231, 233, 236, 237, 238, 242, 232, 244, 243,
       260, 266, 245, 265, 250, 272, 274, 303, 280, 275, 304, 305, 308,
       307, 309, 311, 322, 323, 321, 331, 332, 334, 336, 335, 342, 343,
       350, 345, 338, 361, 360, 365, 371, 366, 374, 373, 372, 380, 375,
       385, 386, 387, 389, 461, 463, 462, 464, 465, 468, 469, 471, 466,
       472, 474, 475, 489, 477, 503, 500, 505, 506, 508, 509, 507, 513,
       512, 502, 516, 515, 521, 518, 522, 519, 523, 524, 528, 529, 531,
       532, 536, 541, 534, 535, 533, 547, 549, 542, 548, 545, 553, 550,
       552, 561, 560, 603, 609, 607, 612, 602, 611, 615, 616, 617, 619,
       622, 621, 600, 623, 624, 628, 629, 625, 627, 630, 631, 632, 633,
       636, 635, 637, 640, 638, 639, 641, 642, 643, 644, 649, 64

In [25]:
import pandas as pd
import geopandas
from shapely.geometry import Point, LineString
from shapely import wkb
import numpy as np
import logging
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from pyproj import CRS as PyprojCRS # IMPORTANTE: Para manipulação de CRS

# Configuração básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

MAX_WORKERS = 4
DEBUG_VERTEX_PROCESSING = False
DEBUG_VERTEX_COUNT = 0

def parse_wkb_hex(wkb_hex_str):
    if pd.isna(wkb_hex_str) or not isinstance(wkb_hex_str, str):
        return None
    try:
        if wkb_hex_str.startswith('0x'):
            wkb_hex_str = wkb_hex_str[2:]
        return wkb.loads(bytes.fromhex(wkb_hex_str))
    except Exception:
        return None

logging.info("Carregando dados das curvas de nível de 'alt_curva_de_nivel.csv'...")
# Nome do arquivo ajustado para o que apareceu no último log.
caminho_arquivo_curvas = 'alt_curva_de_nivel.csv'
gdf_curvas = geopandas.GeoDataFrame()

# !!! IMPORTANTE: IDENTIFIQUE O CRS CORRETO DOS SEUS DADOS DE CURVA DE NÍVEL !!!
# Baseado nos bounds [662031, 7162403 ...], é PROVAVELMENTE um sistema UTM como EPSG:31982.
ORIGINAL_CURVES_CRS_STR = "EPSG:31982" # Ex: SIRGAS 2000 / UTM Zone 22S - VERIFIQUE O SEU!

try:
    df_curvas_raw = pd.read_csv(caminho_arquivo_curvas)
    logging.info(f"Carregado '{caminho_arquivo_curvas}' com {len(df_curvas_raw)} registros de curvas.")

    if 'geom' not in df_curvas_raw.columns or 'elevation' not in df_curvas_raw.columns:
        raise ValueError("Colunas 'geom' ou 'elevation' não encontradas no arquivo de curvas.")

    df_curvas_raw['geometry'] = df_curvas_raw['geom'].apply(parse_wkb_hex)
    df_curvas_raw.dropna(subset=['geometry'], inplace=True)

    if not df_curvas_raw.empty:
        gdf_curvas = geopandas.GeoDataFrame(df_curvas_raw, geometry='geometry', crs=ORIGINAL_CURVES_CRS_STR)
        gdf_curvas['elevation'] = pd.to_numeric(gdf_curvas['elevation'], errors='coerce')
        gdf_curvas.dropna(subset=['elevation'], inplace=True)

        initial_curve_count = len(gdf_curvas)
        gdf_curvas_cleaned = gdf_curvas[gdf_curvas.geometry.is_valid & ~gdf_curvas.geometry.is_empty & gdf_curvas.geometry.notna()].copy()
        logging.info(f"Curvas de nível válidas e não vazias após limpeza inicial: {len(gdf_curvas_cleaned)} (removidas: {initial_curve_count - len(gdf_curvas_cleaned)})")

        if gdf_curvas_cleaned.empty:
            logging.error("GeoDataFrame de curvas de nível está vazio após limpeza.")
        else:
            gdf_curvas = gdf_curvas_cleaned
            logging.info(f"Processadas {len(gdf_curvas)} curvas de nível válidas (CRS original: {gdf_curvas.crs}).")
            logging.info(f"Extensão (bounds) das curvas em {gdf_curvas.crs}: {gdf_curvas.total_bounds}")
            min_elev, max_elev, mean_elev = gdf_curvas['elevation'].min(), gdf_curvas['elevation'].max(), gdf_curvas['elevation'].mean()
            unique_elev_count = gdf_curvas['elevation'].nunique()
            unique_elev_sample = gdf_curvas['elevation'].unique()[:min(10, unique_elev_count)]
            logging.info(f"Estatísticas da 'elevation' das CURVAS: Min={min_elev}, Max={max_elev}, Média={mean_elev:.2f}, Únicas={unique_elev_count}")
            logging.info(f"Amostra de elevações únicas nas curvas: {unique_elev_sample}")
    else:
        logging.error("Nenhuma geometria válida encontrada no arquivo de curvas de nível após o parse.")
except Exception as e:
    logging.error(f"Erro geral ao carregar ou processar dados das curvas de nível: {e}", exc_info=True)

caminho_arquivo_shapes_onibus = 'dados_shapes_onibus_urbs_consolidado_threads.csv'
gdf_bus_routes = geopandas.GeoDataFrame()
try:
    df_bus_shapes_points = pd.read_csv(caminho_arquivo_shapes_onibus)
    # Mantendo o filtro para COD 521 conforme o último log do usuário
    df_bus_shapes_points = df_bus_shapes_points[df_bus_shapes_points['COD'] == 20].reset_index(drop=True)
    logging.info(f"Carregado e filtrado (COD=521) '{caminho_arquivo_shapes_onibus}', resultando em {len(df_bus_shapes_points)} pontos de shapes.")

    df_bus_shapes_points['Latitude'] = pd.to_numeric(df_bus_shapes_points['Latitude'], errors='coerce')
    df_bus_shapes_points['Longitude'] = pd.to_numeric(df_bus_shapes_points['Longitude'], errors='coerce')
    df_bus_shapes_points.dropna(subset=['Latitude', 'Longitude'], inplace=True)
    df_bus_shapes_points['SHP'] = df_bus_shapes_points['SHP'].astype(str)
    df_bus_shapes_points['COD'] = df_bus_shapes_points['COD'].astype(str)

    bus_route_geometries_list = []
    for name, group in df_bus_shapes_points.groupby(['COD', 'SHP'], sort=False):
        if len(group) >= 2:
            line = LineString(zip(group['Longitude'], group['Latitude']))
            bus_route_geometries_list.append({'COD': name[0], 'SHP': name[1], 'geometry': line})
        else:
            logging.warning(f"Rota COD {name[0]}, SHP {name[1]} tem {len(group)} ponto(s), não pode formar LineString.")

    if bus_route_geometries_list:
        gdf_bus_routes = geopandas.GeoDataFrame(bus_route_geometries_list, geometry='geometry', crs="EPSG:4326")
        initial_route_count = len(gdf_bus_routes)
        gdf_bus_routes_cleaned = gdf_bus_routes[gdf_bus_routes.geometry.is_valid & ~gdf_bus_routes.geometry.is_empty & gdf_bus_routes.geometry.notna()].copy()
        logging.info(f"Rotas de ônibus válidas e não vazias após limpeza inicial: {len(gdf_bus_routes_cleaned)} (removidas: {initial_route_count - len(gdf_bus_routes_cleaned)})")
        if gdf_bus_routes_cleaned.empty:
            logging.error("Nenhuma geometria de rota de ônibus válida após limpeza.")
        else:
            gdf_bus_routes = gdf_bus_routes_cleaned
            logging.info(f"Reconstruídas {len(gdf_bus_routes)} geometrias LineString de rotas de ônibus válidas (CRS: {gdf_bus_routes.crs}).")
            logging.info(f"Extensão (bounds) das rotas de ônibus em {gdf_bus_routes.crs}: {gdf_bus_routes.total_bounds}")
    else:
        logging.error("Nenhuma geometria de rota de ônibus pôde ser reconstruída.")
except Exception as e:
    logging.error(f"Erro ao carregar/processar shapes de ônibus: {e}", exc_info=True)

TARGET_PROCESSING_CRS_STR = "EPSG:31982"
TARGET_PROCESSING_CRS_OBJ = PyprojCRS.from_string(TARGET_PROCESSING_CRS_STR) # Objeto CRS para comparação

gdf_curvas_proc = geopandas.GeoDataFrame()
gdf_bus_routes_proc = geopandas.GeoDataFrame()

if not gdf_curvas.empty:
    if gdf_curvas.crs and gdf_curvas.crs.equals(TARGET_PROCESSING_CRS_OBJ): # Comparação correta
        logging.info(f"Curvas de nível já estão no CRS de processamento alvo ({TARGET_PROCESSING_CRS_STR}). Nenhuma reprojeção necessária.")
        gdf_curvas_proc = gdf_curvas.copy()
    elif gdf_curvas.crs:
        logging.info(f"Tentando reprojetar {len(gdf_curvas)} curvas de nível de {gdf_curvas.crs.to_string()} para {TARGET_PROCESSING_CRS_STR}...")
        try:
            gdf_curvas_proc = gdf_curvas.to_crs(TARGET_PROCESSING_CRS_STR)
            initial_count_proj = len(gdf_curvas_proc)
            gdf_curvas_proc = gdf_curvas_proc[gdf_curvas_proc.geometry.is_valid & ~gdf_curvas_proc.geometry.is_empty & gdf_curvas_proc.geometry.notna()].copy()
            cleaned_count_proj = len(gdf_curvas_proc)
            logging.info(f"Curvas de nível reprojetadas. Válidas e não vazias: {cleaned_count_proj} (de {initial_count_proj}).")
            if gdf_curvas_proc.empty and initial_count_proj > 0:
                 logging.warning(f"NENHUMA CURVA DE NÍVEL VÁLIDA RESTOU APÓS REPROJEÇÃO E LIMPEZA.")
        except Exception as e:
            logging.error(f"Falha crítica ao reprojetar gdf_curvas: {e}. gdf_curvas_proc permanecerá vazio.", exc_info=True)
    else:
        logging.error("gdf_curvas não tem CRS definido. Não é possível reprojetar.")
else:
    logging.warning("gdf_curvas original estava vazio.")

if not gdf_bus_routes.empty:
    if gdf_bus_routes.crs and gdf_bus_routes.crs.equals(TARGET_PROCESSING_CRS_OBJ): # Comparação correta
        logging.info(f"Rotas de ônibus já estão no CRS de processamento alvo ({TARGET_PROCESSING_CRS_STR}).")
        gdf_bus_routes_proc = gdf_bus_routes.copy()
    elif gdf_bus_routes.crs:
        logging.info(f"Tentando reprojetar {len(gdf_bus_routes)} rotas de ônibus de {gdf_bus_routes.crs.to_string()} para {TARGET_PROCESSING_CRS_STR}...")
        try:
            gdf_bus_routes_proc = gdf_bus_routes.to_crs(TARGET_PROCESSING_CRS_STR)
            initial_count_proj = len(gdf_bus_routes_proc)
            gdf_bus_routes_proc = gdf_bus_routes_proc[gdf_bus_routes_proc.geometry.is_valid & ~gdf_bus_routes_proc.geometry.is_empty & gdf_bus_routes_proc.geometry.notna()].copy()
            cleaned_count_proj = len(gdf_bus_routes_proc)
            logging.info(f"Rotas de ônibus reprojetadas. Válidas e não vazias: {cleaned_count_proj} (de {initial_count_proj}).")
            if gdf_bus_routes_proc.empty and initial_count_proj > 0:
                logging.warning(f"NENHUMA ROTA DE ÔNIBUS VÁLIDA RESTOU APÓS REPROJEÇÃO E LIMPEZA.")
        except Exception as e:
            logging.error(f"Falha crítica ao reprojetar gdf_bus_routes: {e}. gdf_bus_routes_proc permanecerá vazio.", exc_info=True)
    else:
        logging.error("gdf_bus_routes não tem CRS definido. Não é possível reprojetar.")
else:
    logging.warning("gdf_bus_routes original estava vazio.")

# --- DIAGNÓSTICO DE AMOSTRAS (Corrigido) ---
if not gdf_curvas.empty and gdf_curvas.crs and not gdf_curvas.crs.equals(TARGET_PROCESSING_CRS_OBJ):
    logging.info(f"Diagnóstico de reprojeção para amostras de curvas de nível (de {gdf_curvas.crs.to_string()} para {TARGET_PROCESSING_CRS_STR}):")
    num_samples_to_check = min(3, len(gdf_curvas))
    for i in range(num_samples_to_check):
        # Para testar to_crs em uma única geometria, é melhor criar uma GeoSeries ou GeoDataFrame temporário
        sample_gdf_orig = gdf_curvas.iloc[[i]] # Pega a i-ésima linha como um GeoDataFrame
        sample_geom_orig = sample_gdf_orig.geometry.iloc[0]
        logging.info(f"  Amostra Curva {i} (CRS Original: {sample_gdf_orig.crs.to_string()}): Válida={sample_geom_orig.is_valid}, Vazia={sample_geom_orig.is_empty}, Bounds={sample_geom_orig.bounds}")
        try:
            sample_gdf_proj = sample_gdf_orig.to_crs(TARGET_PROCESSING_CRS_STR) # Reprojeta o GeoDataFrame de uma linha
            sample_geom_proj = sample_gdf_proj.geometry.iloc[0]
            logging.info(f"    -> Amostra Curva {i} (CRS Alvo: {sample_gdf_proj.crs.to_string()}): Válida={sample_geom_proj.is_valid}, Vazia={sample_geom_proj.is_empty}, Bounds={sample_geom_proj.bounds}")
            if not sample_geom_proj.is_valid or sample_geom_proj.is_empty:
                logging.warning(f"      PROBLEMA: Amostra Curva {i} tornou-se inválida/vazia após reprojeção individual.")
        except Exception as e_sample_proj:
            logging.error(f"      ERRO ao reprojetar Amostra Curva {i} individualmente: {e_sample_proj}")


def get_linestring_vertex_elevations(line_geom, gdf_contours_processed, route_cod_shp_for_debug=None):
    global DEBUG_VERTEX_COUNT
    if line_geom is None or line_geom.is_empty or gdf_contours_processed.empty:
        if DEBUG_VERTEX_PROCESSING and DEBUG_VERTEX_COUNT < 5:
            logging.debug(f"DEBUG ({route_cod_shp_for_debug}): get_linestring_vertex_elevations recebeu geometria nula/vazia ou gdf_contours_processed vazio.")
        return []
    
    gdf_contours_valid = gdf_contours_processed[
        gdf_contours_processed.geometry.is_valid & (~gdf_contours_processed.geometry.is_empty)
    ]
    if gdf_contours_valid.empty:
        if DEBUG_VERTEX_PROCESSING and DEBUG_VERTEX_COUNT < 5:
            logging.debug(f"DEBUG ({route_cod_shp_for_debug}): Nenhuma curva de nível VÁLIDA em gdf_contours_processed para comparação.")
        return [] 

    vertex_elevations = []
    coords = []
    if line_geom.geom_type == 'LineString':
        if hasattr(line_geom, 'coords') and len(list(line_geom.coords)) > 0:
             coords.extend(list(line_geom.coords))
    elif line_geom.geom_type == 'MultiLineString':
        for line_segment in line_geom.geoms:
            if hasattr(line_segment, 'coords') and len(list(line_segment.coords)) > 0:
                coords.extend(list(line_segment.coords))
    if not coords:
        if DEBUG_VERTEX_PROCESSING and DEBUG_VERTEX_COUNT < 5:
            logging.debug(f"DEBUG ({route_cod_shp_for_debug}): Nenhuma coordenada extraída da geometria da linha.")
        return []

    for i_coord, coord_pair in enumerate(coords):
        vertex_point = Point(coord_pair)
        if not vertex_point.is_valid or vertex_point.is_empty:
            vertex_elevations.append(np.nan)
            if DEBUG_VERTEX_PROCESSING and DEBUG_VERTEX_COUNT < 5:
                 logging.debug(f"DEBUG ({route_cod_shp_for_debug}) Vértice {i_coord}: Ponto inválido/vazio {vertex_point.wkt}")
            continue
        try:
            distances = gdf_contours_valid.geometry.distance(vertex_point)
            current_debug_active = DEBUG_VERTEX_PROCESSING and DEBUG_VERTEX_COUNT < 2 
            if current_debug_active:
                logging.debug(f"DEBUG ({route_cod_shp_for_debug}) Vértice {i_coord} ({vertex_point.wkt}):")
                logging.debug(f"  Distâncias calculadas (primeiras 5, se houver): {distances.head().tolist() if not distances.empty else 'N/A'}")
                logging.debug(f"  Distances.empty: {distances.empty}, Distances.isna().all(): {distances.isna().all() if not distances.empty else 'N/A'}")

            if not distances.empty and distances.notna().any():
                valid_distances = distances.dropna()
                if not valid_distances.empty:
                    nearest_contour_idx = valid_distances.idxmin()
                    elevation = gdf_contours_valid.loc[nearest_contour_idx, 'elevation']
                    vertex_elevations.append(elevation)
                    if current_debug_active:
                        logging.debug(f"  Vértice {i_coord}: Elev. encontrada {elevation} da curva idx {nearest_contour_idx} (dist: {valid_distances.min():.2f})")
                else:
                    vertex_elevations.append(np.nan) 
                    if current_debug_active: logging.debug(f"  Vértice {i_coord}: Todas as distâncias eram NaN.")
            else:
                vertex_elevations.append(np.nan)
                if current_debug_active: logging.debug(f"  Vértice {i_coord}: Series de distâncias vazia ou todas NaN inicialmente.")
        except Exception as e_dist_loop:
            vertex_elevations.append(np.nan)
            if current_debug_active: logging.debug(f"  Vértice {i_coord}: Exceção no cálculo de distância/elevação: {e_dist_loop}")
        if current_debug_active: DEBUG_VERTEX_COUNT +=1 
    return vertex_elevations

if not gdf_bus_routes_proc.empty and not gdf_curvas_proc.empty:
    logging.info(f"Iniciando cálculo de elevações para os vértices das rotas de ônibus com até {MAX_WORKERS} threads (usando CRS: {gdf_bus_routes_proc.crs.to_string()})...")
    total_routes = len(gdf_bus_routes_proc)
    temp_vertex_elevations_ordered = [None] * total_routes 
    
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        future_to_index = {}
        for original_idx, (_, row_proc) in enumerate(gdf_bus_routes_proc.iterrows()):
            debug_id = f"COD:{row_proc['COD']}-SHP:{row_proc['SHP']}"
            future = executor.submit(get_linestring_vertex_elevations, row_proc['geometry'], gdf_curvas_proc, debug_id)
            future_to_index[future] = original_idx

        processed_count = 0
        for future in as_completed(future_to_index):
            original_idx = future_to_index[future]
            cod_log = gdf_bus_routes.iloc[original_idx]['COD'] 
            shp_log = gdf_bus_routes.iloc[original_idx]['SHP']
            processed_count += 1
            try:
                elevs_result = future.result()
                temp_vertex_elevations_ordered[original_idx] = elevs_result
                if elevs_result and not np.all(np.isnan(elevs_result)): 
                    logging.info(f"Altimetria processada para rota (COD: {cod_log}, SHP: {shp_log}). Progresso: {processed_count}/{total_routes}. Vértices com elevação: {len([e for e in elevs_result if pd.notna(e)])}/{len(elevs_result)}")
                else:
                    logging.info(f"Nenhuma elevação válida encontrada para rota (COD: {cod_log}, SHP: {shp_log}). Progresso: {processed_count}/{total_routes}")
            except Exception as exc:
                temp_vertex_elevations_ordered[original_idx] = [] 
                logging.error(f"Rota (COD: {cod_log}, SHP: {shp_log}) gerou exceção na thread: {exc}", exc_info=False)
                logging.info(f"Progresso: {processed_count}/{total_routes}")

    if len(temp_vertex_elevations_ordered) == len(gdf_bus_routes):
        gdf_bus_routes['vertex_elevations'] = temp_vertex_elevations_ordered
        gdf_bus_routes['min_elevation'] = gdf_bus_routes['vertex_elevations'].apply(lambda x: np.min(x) if x and not np.all(np.isnan(x)) else np.nan)
        gdf_bus_routes['max_elevation'] = gdf_bus_routes['vertex_elevations'].apply(lambda x: np.max(x) if x and not np.all(np.isnan(x)) else np.nan)
        gdf_bus_routes['mean_elevation'] = gdf_bus_routes['vertex_elevations'].apply(lambda x: np.mean(x) if x and not np.all(np.isnan(x)) else np.nan)
        gdf_bus_routes['elevation_diff'] = gdf_bus_routes.apply(
            lambda r: r['max_elevation'] - r['min_elevation'] if pd.notna(r['max_elevation']) and pd.notna(r['min_elevation']) else np.nan, 
            axis=1
        )
        logging.info("Cálculo de elevações (com threads) concluído.")
        print("\nAmostra das rotas de ônibus com dados de altimetria:")
        print(gdf_bus_routes[['COD', 'SHP', 'min_elevation', 'max_elevation', 'mean_elevation', 'elevation_diff']].head())

        gdf_to_save = gdf_bus_routes.drop(columns=['vertex_elevations'], errors='ignore')
        try:
            output_filename = "rotas_onibus_com_altimetria_final.gpkg"
            gdf_to_save.to_file(output_filename, driver="GPKG", layer="rotas_onibus_altimetria")
            logging.info(f"Rotas de ônibus com altimetria salvas em: {output_filename}")
            print(f"\nArquivo salvo: {output_filename}")
        except Exception as e:
            logging.error(f"Erro ao salvar o arquivo GeoPackage: {e}", exc_info=True)
    else:
        logging.error("Discrepância no número de resultados das threads e o GeoDataFrame de rotas.")
elif gdf_bus_routes_proc.empty :
    logging.warning("Nenhuma rota de ônibus válida para processar (após reprojeção/limpeza). Altimetria não calculada.")
elif gdf_curvas_proc.empty :
    logging.warning("Nenhuma curva de nível válida para processar (após reprojeção/limpeza). Não é possível calcular altimetria.")
# Adicione esta importação no início do seu script, se já não estiver lá:
import folium

# ... (seu código existente para criar gdf_bus_routes com as colunas de elevação) ...

# Supondo que gdf_bus_routes já existe (em EPSG:4326) e contém as colunas de elevação
if not gdf_bus_routes.empty and 'mean_elevation' in gdf_bus_routes.columns:
    logging.info("Criando mapa interativo com Folium...")
    try:
        # Folium espera coordenadas em Lat/Lon (EPSG:4326), que é o CRS original de gdf_bus_routes
        # Se gdf_bus_routes não estiver em EPSG:4326, reprojete:
        # gdf_folium = gdf_bus_routes.to_crs("EPSG:4326")
        gdf_folium = gdf_bus_routes # Assumindo que já está em EPSG:4326

        # Calcular um ponto central para o mapa
        if not gdf_folium.geometry.empty:
            map_center_lat = gdf_folium.geometry.unary_union.centroid.y
            map_center_lon = gdf_folium.geometry.unary_union.centroid.x
            
            m = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=12)

            # Adicionar as rotas ao mapa
            # Para colorir, você precisaria de uma lógica para mapear valores para cores
            # Aqui, vamos apenas adicionar com um tooltip
            tooltip_fields = ['COD', 'SHP', 'min_elevation', 'max_elevation', 'mean_elevation', 'elevation_diff']
            # Remove campos do tooltip que não existem no GeoDataFrame
            tooltip_fields = [field for field in tooltip_fields if field in gdf_folium.columns]


            folium.GeoJson(
                gdf_folium,
                name='Rotas de Ônibus com Altimetria',
                style_function=lambda feature: {
                    'color': 'blue', # Cor padrão, pode ser customizada
                    'weight': 3,
                    'opacity': 0.7
                },
                tooltip=folium.GeoJsonTooltip(
                    fields=tooltip_fields,
                    aliases=[f"{field.replace('_', ' ').title()}:" for field in tooltip_fields],
                    sticky=False
                ),
                popup=folium.GeoJsonPopup(
                    fields=tooltip_fields,
                    aliases=[f"{field.replace('_', ' ').title()}:" for field in tooltip_fields],
                )
            ).add_to(m)

            folium.LayerControl().add_to(m)
            
            map_file_name_interactive = "mapa_altimetria_rotas_interativo.html"
            m.save(map_file_name_interactive)
            logging.info(f"Mapa interativo salvo como: {map_file_name_interactive}")
        else:
            logging.warning("Não foi possível criar mapa Folium: geometrias vazias.")
            
    except Exception as e:
        logging.error(f"Erro ao criar mapa interativo: {e}", exc_info=True)
else:
    logging.warning("Não foi possível criar mapa interativo: gdf_bus_routes está vazio ou falta a coluna 'mean_elevation'.")




2025-05-26 17:56:53,005 - INFO - Carregando dados das curvas de nível de 'alt_curva_de_nivel.csv'...
2025-05-26 17:56:55,259 - INFO - Carregado 'alt_curva_de_nivel.csv' com 140426 registros de curvas.
2025-05-26 17:56:56,493 - INFO - Curvas de nível válidas e não vazias após limpeza inicial: 140413 (removidas: 0)
2025-05-26 17:56:56,498 - INFO - Processadas 140413 curvas de nível válidas (CRS original: EPSG:31982).
2025-05-26 17:56:56,513 - INFO - Extensão (bounds) das curvas em EPSG:31982: [ 662031.9262085  7162403.66021729  682796.32299805 7195629.39019775]
2025-05-26 17:56:56,518 - INFO - Estatísticas da 'elevation' das CURVAS: Min=862.0, Max=1020.0, Média=904.22, Únicas=154
2025-05-26 17:56:56,519 - INFO - Amostra de elevações únicas nas curvas: [909. 911. 912. 921. 913. 873. 914. 916. 981. 982.]
2025-05-26 17:56:56,613 - INFO - Carregado e filtrado (COD=521) 'dados_shapes_onibus_urbs_consolidado_threads.csv', resultando em 981 pontos de shapes.
2025-05-26 17:56:56,621 - INFO - Rot


Amostra das rotas de ônibus com dados de altimetria:
  COD   SHP  min_elevation  max_elevation  mean_elevation  elevation_diff
0  20  4091          875.0          990.0      926.989806           115.0

Arquivo salvo: rotas_onibus_com_altimetria_final.gpkg


In [23]:
gdf_to_save

,COD,SHP,geometry,min_elevation,max_elevation,mean_elevation,elevation_diff
0,22,4146,"LINESTRING (-49.25216 -25.40663, -49.25203 -25...",875.0,966.0,916.646454,91.0


In [6]:
import pandas as pd
import geopandas
from shapely.geometry import Point, LineString # Mantido como no seu script
from shapely import wkb
import numpy as np
import logging
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from pyproj import CRS as PyprojCRS # IMPORTANTE: Para manipulação de CRS

# Configuração básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

MAX_WORKERS = 4
DEBUG_VERTEX_PROCESSING = False
DEBUG_VERTEX_COUNT = 0

def parse_wkb_hex(wkb_hex_str):
    if pd.isna(wkb_hex_str) or not isinstance(wkb_hex_str, str):
        return None
    try:
        if wkb_hex_str.startswith('0x'):
            wkb_hex_str = wkb_hex_str[2:]
        return wkb.loads(bytes.fromhex(wkb_hex_str))
    except Exception:
        return None

logging.info("Carregando dados das curvas de nível de 'alt_curva_de_nivel.csv'...")
# Nome do arquivo ajustado para o que apareceu no último log.
caminho_arquivo_curvas = 'alt_curva_de_nivel.csv'
gdf_curvas = geopandas.GeoDataFrame()

# !!! IMPORTANTE: IDENTIFIQUE O CRS CORRETO DOS SEUS DADOS DE CURVA DE NÍVEL !!!
# Baseado nos bounds [662031, 7162403 ...], é PROVAVELMENTE um sistema UTM como EPSG:31982.
ORIGINAL_CURVES_CRS_STR = "EPSG:31982" # Ex: SIRGAS 2000 / UTM Zone 22S - VERIFIQUE O SEU!

try:
    df_curvas_raw = pd.read_csv(caminho_arquivo_curvas)
    logging.info(f"Carregado '{caminho_arquivo_curvas}' com {len(df_curvas_raw)} registros de curvas.")

    if 'geom' not in df_curvas_raw.columns or 'elevation' not in df_curvas_raw.columns:
        raise ValueError("Colunas 'geom' ou 'elevation' não encontradas no arquivo de curvas.")

    df_curvas_raw['geometry'] = df_curvas_raw['geom'].apply(parse_wkb_hex)
    df_curvas_raw.dropna(subset=['geometry'], inplace=True)

    if not df_curvas_raw.empty:
        gdf_curvas = geopandas.GeoDataFrame(df_curvas_raw, geometry='geometry', crs=ORIGINAL_CURVES_CRS_STR)
        gdf_curvas['elevation'] = pd.to_numeric(gdf_curvas['elevation'], errors='coerce')
        gdf_curvas.dropna(subset=['elevation'], inplace=True)

        initial_curve_count = len(gdf_curvas)
        gdf_curvas_cleaned = gdf_curvas[gdf_curvas.geometry.is_valid & ~gdf_curvas.geometry.is_empty & gdf_curvas.geometry.notna()].copy()
        logging.info(f"Curvas de nível válidas e não vazias após limpeza inicial: {len(gdf_curvas_cleaned)} (removidas: {initial_curve_count - len(gdf_curvas_cleaned)})")

        if gdf_curvas_cleaned.empty:
            logging.error("GeoDataFrame de curvas de nível está vazio após limpeza.")
        else:
            gdf_curvas = gdf_curvas_cleaned
            logging.info(f"Processadas {len(gdf_curvas)} curvas de nível válidas (CRS original: {gdf_curvas.crs}).")
            logging.info(f"Extensão (bounds) das curvas em {gdf_curvas.crs}: {gdf_curvas.total_bounds}")
            min_elev, max_elev, mean_elev = gdf_curvas['elevation'].min(), gdf_curvas['elevation'].max(), gdf_curvas['elevation'].mean()
            unique_elev_count = gdf_curvas['elevation'].nunique()
            unique_elev_sample = gdf_curvas['elevation'].unique()[:min(10, unique_elev_count)]
            logging.info(f"Estatísticas da 'elevation' das CURVAS: Min={min_elev}, Max={max_elev}, Média={mean_elev:.2f}, Únicas={unique_elev_count}")
            logging.info(f"Amostra de elevações únicas nas curvas: {unique_elev_sample}")
    else:
        logging.error("Nenhuma geometria válida encontrada no arquivo de curvas de nível após o parse.")
except Exception as e:
    logging.error(f"Erro geral ao carregar ou processar dados das curvas de nível: {e}", exc_info=True)

caminho_arquivo_shapes_onibus = 'dados_shapes_onibus_urbs_consolidado_threads.csv'
gdf_bus_routes = geopandas.GeoDataFrame()
try:
    df_bus_shapes_points = pd.read_csv(caminho_arquivo_shapes_onibus)
    # Mantendo o filtro para COD 20 conforme o último log do usuário (ajustado de 521 para 20)
    df_bus_shapes_points = df_bus_shapes_points[df_bus_shapes_points['COD'] == 22].reset_index(drop=True)
    logging.info(f"Carregado e filtrado (COD=20) '{caminho_arquivo_shapes_onibus}', resultando em {len(df_bus_shapes_points)} pontos de shapes.")

    df_bus_shapes_points['Latitude'] = pd.to_numeric(df_bus_shapes_points['Latitude'], errors='coerce')
    df_bus_shapes_points['Longitude'] = pd.to_numeric(df_bus_shapes_points['Longitude'], errors='coerce')
    df_bus_shapes_points.dropna(subset=['Latitude', 'Longitude'], inplace=True)
    df_bus_shapes_points['SHP'] = df_bus_shapes_points['SHP'].astype(str)
    df_bus_shapes_points['COD'] = df_bus_shapes_points['COD'].astype(str)

    bus_route_geometries_list = []
    for name, group in df_bus_shapes_points.groupby(['COD', 'SHP'], sort=False):
        if len(group) >= 2:
            line = LineString(zip(group['Longitude'], group['Latitude']))
            bus_route_geometries_list.append({'COD': name[0], 'SHP': name[1], 'geometry': line})
        else:
            logging.warning(f"Rota COD {name[0]}, SHP {name[1]} tem {len(group)} ponto(s), não pode formar LineString.")

    if bus_route_geometries_list:
        gdf_bus_routes = geopandas.GeoDataFrame(bus_route_geometries_list, geometry='geometry', crs="EPSG:4326")
        initial_route_count = len(gdf_bus_routes)
        gdf_bus_routes_cleaned = gdf_bus_routes[gdf_bus_routes.geometry.is_valid & ~gdf_bus_routes.geometry.is_empty & gdf_bus_routes.geometry.notna()].copy()
        logging.info(f"Rotas de ônibus válidas e não vazias após limpeza inicial: {len(gdf_bus_routes_cleaned)} (removidas: {initial_route_count - len(gdf_bus_routes_cleaned)})")
        if gdf_bus_routes_cleaned.empty:
            logging.error("Nenhuma geometria de rota de ônibus válida após limpeza.")
        else:
            gdf_bus_routes = gdf_bus_routes_cleaned
            logging.info(f"Reconstruídas {len(gdf_bus_routes)} geometrias LineString de rotas de ônibus válidas (CRS: {gdf_bus_routes.crs}).")
            logging.info(f"Extensão (bounds) das rotas de ônibus em {gdf_bus_routes.crs}: {gdf_bus_routes.total_bounds}")
    else:
        logging.error("Nenhuma geometria de rota de ônibus pôde ser reconstruída.")
except Exception as e:
    logging.error(f"Erro ao carregar/processar shapes de ônibus: {e}", exc_info=True)

TARGET_PROCESSING_CRS_STR = "EPSG:31982"
TARGET_PROCESSING_CRS_OBJ = PyprojCRS.from_string(TARGET_PROCESSING_CRS_STR) # Objeto CRS para comparação

gdf_curvas_proc = geopandas.GeoDataFrame()
gdf_bus_routes_proc = geopandas.GeoDataFrame()

if not gdf_curvas.empty:
    if gdf_curvas.crs and gdf_curvas.crs.equals(TARGET_PROCESSING_CRS_OBJ): # Comparação correta
        logging.info(f"Curvas de nível já estão no CRS de processamento alvo ({TARGET_PROCESSING_CRS_STR}). Nenhuma reprojeção necessária.")
        gdf_curvas_proc = gdf_curvas.copy()
    elif gdf_curvas.crs:
        logging.info(f"Tentando reprojetar {len(gdf_curvas)} curvas de nível de {gdf_curvas.crs.to_string()} para {TARGET_PROCESSING_CRS_STR}...")
        try:
            gdf_curvas_proc = gdf_curvas.to_crs(TARGET_PROCESSING_CRS_STR)
            initial_count_proj = len(gdf_curvas_proc)
            gdf_curvas_proc = gdf_curvas_proc[gdf_curvas_proc.geometry.is_valid & ~gdf_curvas_proc.geometry.is_empty & gdf_curvas_proc.geometry.notna()].copy()
            cleaned_count_proj = len(gdf_curvas_proc)
            logging.info(f"Curvas de nível reprojetadas. Válidas e não vazias: {cleaned_count_proj} (de {initial_count_proj}).")
            if gdf_curvas_proc.empty and initial_count_proj > 0:
                logging.warning(f"NENHUMA CURVA DE NÍVEL VÁLIDA RESTOU APÓS REPROJEÇÃO E LIMPEZA.")
        except Exception as e:
            logging.error(f"Falha crítica ao reprojetar gdf_curvas: {e}. gdf_curvas_proc permanecerá vazio.", exc_info=True)
    else:
        logging.error("gdf_curvas não tem CRS definido. Não é possível reprojetar.")
else:
    logging.warning("gdf_curvas original estava vazio.")

if not gdf_bus_routes.empty:
    if gdf_bus_routes.crs and gdf_bus_routes.crs.equals(TARGET_PROCESSING_CRS_OBJ): # Comparação correta
        logging.info(f"Rotas de ônibus já estão no CRS de processamento alvo ({TARGET_PROCESSING_CRS_STR}).")
        gdf_bus_routes_proc = gdf_bus_routes.copy()
    elif gdf_bus_routes.crs:
        logging.info(f"Tentando reprojetar {len(gdf_bus_routes)} rotas de ônibus de {gdf_bus_routes.crs.to_string()} para {TARGET_PROCESSING_CRS_STR}...")
        try:
            gdf_bus_routes_proc = gdf_bus_routes.to_crs(TARGET_PROCESSING_CRS_STR)
            initial_count_proj = len(gdf_bus_routes_proc)
            gdf_bus_routes_proc = gdf_bus_routes_proc[gdf_bus_routes_proc.geometry.is_valid & ~gdf_bus_routes_proc.geometry.is_empty & gdf_bus_routes_proc.geometry.notna()].copy()
            cleaned_count_proj = len(gdf_bus_routes_proc)
            logging.info(f"Rotas de ônibus reprojetadas. Válidas e não vazias: {cleaned_count_proj} (de {initial_count_proj}).")
            if gdf_bus_routes_proc.empty and initial_count_proj > 0:
                logging.warning(f"NENHUMA ROTA DE ÔNIBUS VÁLIDA RESTOU APÓS REPROJEÇÃO E LIMPEZA.")
        except Exception as e:
            logging.error(f"Falha crítica ao reprojetar gdf_bus_routes: {e}. gdf_bus_routes_proc permanecerá vazio.", exc_info=True)
    else:
        logging.error("gdf_bus_routes não tem CRS definido. Não é possível reprojetar.")
else:
    logging.warning("gdf_bus_routes original estava vazio.")

# --- DIAGNÓSTICO DE AMOSTRAS (Corrigido) ---
if not gdf_curvas.empty and gdf_curvas.crs and not gdf_curvas.crs.equals(TARGET_PROCESSING_CRS_OBJ):
    logging.info(f"Diagnóstico de reprojeção para amostras de curvas de nível (de {gdf_curvas.crs.to_string()} para {TARGET_PROCESSING_CRS_STR}):")
    num_samples_to_check = min(3, len(gdf_curvas))
    for i in range(num_samples_to_check):
        sample_gdf_orig = gdf_curvas.iloc[[i]] 
        sample_geom_orig = sample_gdf_orig.geometry.iloc[0]
        logging.info(f"  Amostra Curva {i} (CRS Original: {sample_gdf_orig.crs.to_string()}): Válida={sample_geom_orig.is_valid}, Vazia={sample_geom_orig.is_empty}, Bounds={sample_geom_orig.bounds}")
        try:
            sample_gdf_proj = sample_gdf_orig.to_crs(TARGET_PROCESSING_CRS_STR) 
            sample_geom_proj = sample_gdf_proj.geometry.iloc[0]
            logging.info(f"    -> Amostra Curva {i} (CRS Alvo: {sample_gdf_proj.crs.to_string()}): Válida={sample_geom_proj.is_valid}, Vazia={sample_geom_proj.is_empty}, Bounds={sample_geom_proj.bounds}")
            if not sample_geom_proj.is_valid or sample_geom_proj.is_empty:
                logging.warning(f"      PROBLEMA: Amostra Curva {i} tornou-se inválida/vazia após reprojeção individual.")
        except Exception as e_sample_proj:
            logging.error(f"      ERRO ao reprojetar Amostra Curva {i} individualmente: {e_sample_proj}")

# --- Funções Adicionadas para Cálculo de Ganho/Perda Cumulativa ---
def _calculate_cumulative_elevation_gain(elevations):
    """Calcula o ganho de elevação cumulativo a partir de uma lista de elevações."""
    if not isinstance(elevations, list) or not elevations:
        return np.nan

    valid_elevations = [e for e in elevations if pd.notna(e) and isinstance(e, (int, float))]

    if len(valid_elevations) < 2:
        return np.nan

    cumulative_gain = 0.0
    for i in range(1, len(valid_elevations)):
        diff = valid_elevations[i] - valid_elevations[i-1]
        if diff > 0:
            cumulative_gain += diff
    return cumulative_gain

def _calculate_cumulative_elevation_loss(elevations):
    """Calcula a perda de elevação cumulativa a partir de uma lista de elevações."""
    if not isinstance(elevations, list) or not elevations:
        return np.nan

    valid_elevations = [e for e in elevations if pd.notna(e) and isinstance(e, (int, float))]

    if len(valid_elevations) < 2:
        return np.nan

    cumulative_loss = 0.0
    for i in range(1, len(valid_elevations)):
        diff = valid_elevations[i] - valid_elevations[i-1]
        if diff < 0:
            cumulative_loss += abs(diff)
    return cumulative_loss
# --- Fim das Funções Adicionadas ---

def get_linestring_vertex_elevations(line_geom, gdf_contours_processed, route_cod_shp_for_debug=None):
    global DEBUG_VERTEX_COUNT
    if line_geom is None or line_geom.is_empty or gdf_contours_processed.empty:
        if DEBUG_VERTEX_PROCESSING and DEBUG_VERTEX_COUNT < 5:
            logging.debug(f"DEBUG ({route_cod_shp_for_debug}): get_linestring_vertex_elevations recebeu geometria nula/vazia ou gdf_contours_processed vazio.")
        return [] # Retorna lista vazia se não há geometria ou contornos
    
    # Filtra por geometrias válidas e não vazias DENTRO da função para garantir que sempre trabalhe com dados limpos
    gdf_contours_valid = gdf_contours_processed[
        gdf_contours_processed.geometry.is_valid & (~gdf_contours_processed.geometry.is_empty)
    ]
    if gdf_contours_valid.empty:
        if DEBUG_VERTEX_PROCESSING and DEBUG_VERTEX_COUNT < 5: # Limita o número de logs para não poluir
            logging.debug(f"DEBUG ({route_cod_shp_for_debug}): Nenhuma curva de nível VÁLIDA em gdf_contours_processed para comparação.")
        return [] # Retorna lista vazia se não há contornos válidos

    vertex_elevations = []
    coords = []
    # Trata LineString e MultiLineString para extrair coordenadas
    if line_geom.geom_type == 'LineString':
        if hasattr(line_geom, 'coords') and len(list(line_geom.coords)) > 0: # Verifica se há coordenadas
            coords.extend(list(line_geom.coords))
    elif line_geom.geom_type == 'MultiLineString': # Supondo que MultiLineString foi importado ou tratado
        for line_segment in line_geom.geoms:
            if hasattr(line_segment, 'coords') and len(list(line_segment.coords)) > 0:
                coords.extend(list(line_segment.coords))
    
    if not coords: # Se nenhuma coordenada foi extraída
        if DEBUG_VERTEX_PROCESSING and DEBUG_VERTEX_COUNT < 5:
            logging.debug(f"DEBUG ({route_cod_shp_for_debug}): Nenhuma coordenada extraída da geometria da linha.")
        return []

    for i_coord, coord_pair in enumerate(coords):
        vertex_point = Point(coord_pair)
        if not vertex_point.is_valid or vertex_point.is_empty:
            vertex_elevations.append(np.nan)
            if DEBUG_VERTEX_PROCESSING and DEBUG_VERTEX_COUNT < 5: # Limita logs
                logging.debug(f"DEBUG ({route_cod_shp_for_debug}) Vértice {i_coord}: Ponto inválido/vazio {vertex_point.wkt}")
            continue
        try:
            distances = gdf_contours_valid.geometry.distance(vertex_point)
            current_debug_active = DEBUG_VERTEX_PROCESSING and DEBUG_VERTEX_COUNT < 2 # Debug mais verboso para os primeiros 2 vértices processados globalmente
            
            if current_debug_active: # Logging de debug mais detalhado
                logging.debug(f"DEBUG ({route_cod_shp_for_debug}) Vértice {i_coord} ({vertex_point.wkt}):")
                logging.debug(f"  Distâncias calculadas (primeiras 5, se houver): {distances.head().tolist() if not distances.empty else 'N/A'}")
                logging.debug(f"  Distances.empty: {distances.empty}, Distances.isna().all(): {distances.isna().all() if not distances.empty else 'N/A'}")

            if not distances.empty and distances.notna().any(): # Verifica se há distâncias válidas
                valid_distances = distances.dropna() # Remove NaNs das distâncias
                if not valid_distances.empty:
                    nearest_contour_idx = valid_distances.idxmin() # Índice da curva mais próxima
                    elevation = gdf_contours_valid.loc[nearest_contour_idx, 'elevation']
                    vertex_elevations.append(elevation)
                    if current_debug_active:
                        logging.debug(f"  Vértice {i_coord}: Elev. encontrada {elevation} da curva idx {nearest_contour_idx} (dist: {valid_distances.min():.2f})")
                else: # Caso raro onde todas as distâncias calculadas são NaN (ex: ponto muito distante)
                    vertex_elevations.append(np.nan) 
                    if current_debug_active: logging.debug(f"  Vértice {i_coord}: Todas as distâncias eram NaN.")
            else: # Se a série de distâncias estiver vazia ou todas as distâncias forem NaN
                vertex_elevations.append(np.nan)
                if current_debug_active: logging.debug(f"  Vértice {i_coord}: Series de distâncias vazia ou todas NaN inicialmente.")
        except Exception as e_dist_loop:
            vertex_elevations.append(np.nan)
            if current_debug_active: logging.debug(f"  Vértice {i_coord}: Exceção no cálculo de distância/elevação: {e_dist_loop}")
        
        if current_debug_active: DEBUG_VERTEX_COUNT +=1 
    return vertex_elevations

if not gdf_bus_routes_proc.empty and not gdf_curvas_proc.empty:
    logging.info(f"Iniciando cálculo de elevações para os vértices das rotas de ônibus com até {MAX_WORKERS} threads (usando CRS: {gdf_bus_routes_proc.crs.to_string()})...")
    total_routes = len(gdf_bus_routes_proc)
    temp_vertex_elevations_ordered = [None] * total_routes 
    
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        future_to_index = {}
        # Usar o índice original de gdf_bus_routes (que corresponde a gdf_bus_routes_proc se não houve filtro que mude a ordem)
        for original_idx, (_, row_proc) in enumerate(gdf_bus_routes_proc.iterrows()):
            # Para o debug ID, usamos os dados da linha processada, pois o índice de gdf_bus_routes pode não alinhar se houve descarte
            debug_id = f"COD:{row_proc['COD']}-SHP:{row_proc['SHP']}"
            future = executor.submit(get_linestring_vertex_elevations, row_proc['geometry'], gdf_curvas_proc, debug_id)
            future_to_index[future] = original_idx # Mapeia o futuro para o índice posicional em temp_vertex_elevations_ordered

        processed_count = 0
        for future in as_completed(future_to_index):
            original_idx = future_to_index[future] # Este é o índice para temp_vertex_elevations_ordered e gdf_bus_routes
            # Para logging, pegar COD/SHP de gdf_bus_routes original usando o índice original que foi preservado.
            cod_log = gdf_bus_routes.iloc[original_idx]['COD'] 
            shp_log = gdf_bus_routes.iloc[original_idx]['SHP']
            processed_count += 1
            try:
                elevs_result = future.result()
                temp_vertex_elevations_ordered[original_idx] = elevs_result
                
                # Verifica se elevs_result não é None e contém alguma elevação não-NaN
                if elevs_result and not np.all(np.isnan(elevs_result)): 
                    logging.info(f"Altimetria processada para rota (COD: {cod_log}, SHP: {shp_log}). Progresso: {processed_count}/{total_routes}. Vértices com elevação: {len([e for e in elevs_result if pd.notna(e)])}/{len(elevs_result)}")
                else:
                    logging.info(f"Nenhuma elevação válida encontrada para rota (COD: {cod_log}, SHP: {shp_log}). Progresso: {processed_count}/{total_routes}")
            except Exception as exc:
                temp_vertex_elevations_ordered[original_idx] = [] # Lista vazia em caso de erro na thread
                logging.error(f"Rota (COD: {cod_log}, SHP: {shp_log}) gerou exceção na thread: {exc}", exc_info=False)
                logging.info(f"Progresso: {processed_count}/{total_routes}") # Log de progresso mesmo em erro

    # Atribuir os resultados de volta ao gdf_bus_routes original (que está em EPSG:4326)
    if len(temp_vertex_elevations_ordered) == len(gdf_bus_routes):
        gdf_bus_routes['vertex_elevations'] = temp_vertex_elevations_ordered
        # Corrigido para usar np.nanmin, np.nanmax, np.nanmean que ignoram NaNs corretamente.
        # A lambda precisa verificar se x é uma lista e se contém algum valor não-NaN.
        gdf_bus_routes['min_elevation'] = gdf_bus_routes['vertex_elevations'].apply(
            lambda x: np.nanmin([val for val in x if pd.notna(val)]) if isinstance(x, list) and any(pd.notna(val) for val in x) else np.nan)
        gdf_bus_routes['max_elevation'] = gdf_bus_routes['vertex_elevations'].apply(
            lambda x: np.nanmax([val for val in x if pd.notna(val)]) if isinstance(x, list) and any(pd.notna(val) for val in x) else np.nan)
        gdf_bus_routes['mean_elevation'] = gdf_bus_routes['vertex_elevations'].apply(
            lambda x: np.nanmean([val for val in x if pd.notna(val)]) if isinstance(x, list) and any(pd.notna(val) for val in x) else np.nan)
        
        gdf_bus_routes['elevation_diff'] = gdf_bus_routes.apply(
            lambda r: r['max_elevation'] - r['min_elevation'] if pd.notna(r['max_elevation']) and pd.notna(r['min_elevation']) else np.nan, 
            axis=1
        )
        
        # --- Adição do cálculo de ganho/perda cumulativa ---
        gdf_bus_routes['cumulative_gain'] = gdf_bus_routes['vertex_elevations'].apply(_calculate_cumulative_elevation_gain)
        gdf_bus_routes['cumulative_loss'] = gdf_bus_routes['vertex_elevations'].apply(_calculate_cumulative_elevation_loss)
        # --- Fim da adição ---

        logging.info("Cálculo de elevações (com threads) concluído.")
        print("\nAmostra das rotas de ônibus com dados de altimetria:")
        # A linha abaixo permanece inalterada conforme a solicitação
        print(gdf_bus_routes[['COD', 'SHP', 'min_elevation', 'max_elevation', 'mean_elevation', 'elevation_diff']].head())

        gdf_to_save = gdf_bus_routes.drop(columns=['vertex_elevations'], errors='ignore')
        try:
            output_filename = "rotas_onibus_com_altimetria_final.gpkg"
            gdf_to_save.to_file(output_filename, driver="GPKG", layer="rotas_onibus_altimetria")
            logging.info(f"Rotas de ônibus com altimetria salvas em: {output_filename}")
            print(f"\nArquivo salvo: {output_filename}")
        except Exception as e:
            logging.error(f"Erro ao salvar o arquivo GeoPackage: {e}", exc_info=True)
    else:
        logging.error("Discrepância no número de resultados das threads e o GeoDataFrame de rotas.")
elif gdf_bus_routes_proc.empty :
    logging.warning("Nenhuma rota de ônibus válida para processar (após reprojeção/limpeza). Altimetria não calculada.")
elif gdf_curvas_proc.empty :
    logging.warning("Nenhuma curva de nível válida para processar (após reprojeção/limpeza). Não é possível calcular altimetria.")

# Adicione esta importação no início do seu script, se já não estiver lá:
import folium # Já importado no início do script original completo, mas garantindo aqui.

# ... (seu código existente para criar gdf_bus_routes com as colunas de elevação) ...

# Supondo que gdf_bus_routes já existe (em EPSG:4326) e contém as colunas de elevação
if not gdf_bus_routes.empty and 'mean_elevation' in gdf_bus_routes.columns:
    logging.info("Criando mapa interativo com Folium...")
    try:
        gdf_folium = gdf_bus_routes 

        if not gdf_folium.geometry.empty:
            # Tenta calcular o centroide de forma mais robusta, ignorando geometrias vazias/inválidas
            valid_geoms_for_centroid = gdf_folium[gdf_folium.geometry.is_valid & ~gdf_folium.geometry.is_empty]
            if not valid_geoms_for_centroid.empty:
                 map_center_lat = valid_geoms_for_centroid.geometry.unary_union.centroid.y
                 map_center_lon = valid_geoms_for_centroid.geometry.unary_union.centroid.x
            else: # Fallback se não houver geometrias válidas
                logging.warning("Nenhuma geometria válida para calcular o centroide do mapa Folium. Usando (0,0).")
                map_center_lat, map_center_lon = 0,0
            
            m = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=12)

            tooltip_fields = ['COD', 'SHP', 'min_elevation', 'max_elevation', 'mean_elevation', 'elevation_diff']
            tooltip_fields = [field for field in tooltip_fields if field in gdf_folium.columns]


            folium.GeoJson(
                gdf_folium,
                name='Rotas de Ônibus com Altimetria',
                style_function=lambda feature: {
                    'color': 'blue', 
                    'weight': 3,
                    'opacity': 0.7
                },
                tooltip=folium.GeoJsonTooltip(
                    fields=tooltip_fields,
                    aliases=[f"{field.replace('_', ' ').title()}:" for field in tooltip_fields],
                    sticky=False,
                    localize=True # Para melhor formatação de números
                ),
                popup=folium.GeoJsonPopup(
                    fields=tooltip_fields,
                    aliases=[f"{field.replace('_', ' ').title()}:" for field in tooltip_fields],
                    localize=True
                )
            ).add_to(m)

            folium.LayerControl().add_to(m)
            
            map_file_name_interactive = "mapa_altimetria_rotas_interativo.html"
            m.save(map_file_name_interactive)
            logging.info(f"Mapa interativo salvo como: {map_file_name_interactive}")
        else:
            logging.warning("Não foi possível criar mapa Folium: geometrias vazias em gdf_folium.")
            
    except Exception as e:
        logging.error(f"Erro ao criar mapa interativo: {e}", exc_info=True)
else:
    logging.warning("Não foi possível criar mapa interativo: gdf_bus_routes está vazio ou falta a coluna 'mean_elevation'.")

2025-05-27 09:28:23,258 - INFO - Carregando dados das curvas de nível de 'alt_curva_de_nivel.csv'...
2025-05-27 09:28:25,806 - INFO - Carregado 'alt_curva_de_nivel.csv' com 140426 registros de curvas.
2025-05-27 09:28:27,247 - INFO - Curvas de nível válidas e não vazias após limpeza inicial: 140413 (removidas: 0)
2025-05-27 09:28:27,251 - INFO - Processadas 140413 curvas de nível válidas (CRS original: EPSG:31982).
2025-05-27 09:28:27,267 - INFO - Extensão (bounds) das curvas em EPSG:31982: [ 662031.9262085  7162403.66021729  682796.32299805 7195629.39019775]
2025-05-27 09:28:27,272 - INFO - Estatísticas da 'elevation' das CURVAS: Min=862.0, Max=1020.0, Média=904.22, Únicas=154
2025-05-27 09:28:27,273 - INFO - Amostra de elevações únicas nas curvas: [909. 911. 912. 921. 913. 873. 914. 916. 981. 982.]
2025-05-27 09:28:27,370 - INFO - Carregado e filtrado (COD=20) 'dados_shapes_onibus_urbs_consolidado_threads.csv', resultando em 973 pontos de shapes.
2025-05-27 09:28:27,376 - INFO - Rota


Amostra das rotas de ônibus com dados de altimetria:
  COD   SHP  min_elevation  max_elevation  mean_elevation  elevation_diff
0  22  4146          875.0          966.0      916.646454            91.0

Arquivo salvo: rotas_onibus_com_altimetria_final.gpkg


In [7]:
gdf_bus_routes

,COD,SHP,geometry,vertex_elevations,min_elevation,max_elevation,mean_elevation,elevation_diff,cumulative_gain,cumulative_loss
0,22,4146,"LINESTRING (-49.25216 -25.40663, -49.25203 -25...","[912.0, 912.0, 911.0, 910.0, 910.0, 909.0, 908...",875.0,966.0,916.646454,91.0,612.0,612.0
